In [1]:
# Cell 1: Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy import stats
import json
import os
from pathlib import Path
from sklearn.inspection import permutation_importance

# Visualization settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
sns.set_palette("viridis")

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
import xgboost as xgb
from datetime import datetime

# # Set visualization style
# plt.style.use('seaborn')
# sns.set_palette("husl")

In [2]:
def create_lag_features(data, target_col='AQI', n_lags=7):
    """Create lag features for the target column"""
    data = data.sort_values('Date')
    for i in range(1, n_lags + 1):
        data[f'{target_col}_lag_{i}'] = data[target_col].shift(i)
    return data

# Cell 2: Load pre-split data for each city
processed_dir = '../data/processed'
cities = ['bengaluru', 'chennai', 'delhi', 'hyderabad']

# Dictionary to store data for each city
city_data = {}

for city in cities:
    city_dir = f'{processed_dir}/{city.lower()}'
    city_data[city] = {
        'train': pd.read_csv(f'{city_dir}/train.csv'),
        'val': pd.read_csv(f'{city_dir}/val.csv'),
        'test': pd.read_csv(f'{city_dir}/test.csv')
    }
    
    # Convert date columns to datetime
    for split in ['train', 'val', 'test']:
        city_data[city][split]['Date'] = pd.to_datetime(city_data[city][split]['Date'])
        
        # Check if lag features exist, if not create them
        if 'AQI_lag_1' not in city_data[city][split].columns:
            print(f"Creating lag features for {city} {split} set")
            city_data[city][split] = create_lag_features(city_data[city][split])
    
    print(f"\n{city.title()} data loaded:")
    print(f"Train: {city_data[city]['train'].shape[0]} samples")
    print(f"Validation: {city_data[city]['val'].shape[0]} samples")
    print(f"Test: {city_data[city]['test'].shape[0]} samples")

Creating lag features for bengaluru train set
Creating lag features for bengaluru val set
Creating lag features for bengaluru test set

Bengaluru data loaded:
Train: 1241 samples
Validation: 287 samples
Test: 382 samples
Creating lag features for chennai train set
Creating lag features for chennai val set
Creating lag features for chennai test set

Chennai data loaded:
Train: 1224 samples
Validation: 283 samples
Test: 377 samples
Creating lag features for delhi train set
Creating lag features for delhi val set
Creating lag features for delhi test set

Delhi data loaded:
Train: 1299 samples
Validation: 300 samples
Test: 400 samples
Creating lag features for hyderabad train set
Creating lag features for hyderabad val set
Creating lag features for hyderabad test set

Hyderabad data loaded:
Train: 1222 samples
Validation: 282 samples
Test: 376 samples


In [3]:
# Cell 3: Define utility functions
def calculate_metrics(y_true, y_pred, prefix=''):
    """Calculate comprehensive evaluation metrics with confidence intervals"""
    metrics = {}
    
    # Remove any NaN values from both arrays
    mask = ~(np.isnan(y_true) | np.isnan(y_pred))
    y_true = y_true[mask]
    y_pred = y_pred[mask]
    
    if len(y_true) == 0 or len(y_pred) == 0:
        print("Warning: No valid data points after removing NaN values")
        return {f'{prefix}rmse': np.nan, f'{prefix}mae': np.nan, f'{prefix}r2': np.nan}
    
    # Calculate basic metrics
    metrics[f'{prefix}rmse'] = np.sqrt(mean_squared_error(y_true, y_pred))
    metrics[f'{prefix}mae'] = mean_absolute_error(y_true, y_pred)
    metrics[f'{prefix}r2'] = r2_score(y_true, y_pred)
    
    # Calculate confidence intervals using bootstrap
    n_iterations = 1000
    n_samples = len(y_true)
    
    # Initialize arrays to store bootstrap metrics
    rmse_boots = np.zeros(n_iterations)
    mae_boots = np.zeros(n_iterations)
    r2_boots = np.zeros(n_iterations)
    
    for i in range(n_iterations):
        # Sample with replacement
        indices = np.random.randint(0, n_samples, n_samples)
        y_true_boot = y_true[indices]
        y_pred_boot = y_pred[indices]
        
        # Calculate metrics for this bootstrap sample
        rmse_boots[i] = np.sqrt(mean_squared_error(y_true_boot, y_pred_boot))
        mae_boots[i] = mean_absolute_error(y_true_boot, y_pred_boot)
        r2_boots[i] = r2_score(y_true_boot, y_pred_boot)
    
    # Calculate 95% confidence intervals
    for metric_name, metric_boots in [('rmse', rmse_boots), ('mae', mae_boots), ('r2', r2_boots)]:
        lower, upper = np.percentile(metric_boots, [2.5, 97.5])
        metrics[f'{prefix}{metric_name}_ci'] = (lower, upper)
    
    return metrics

def print_metrics(metrics):
    """Print metrics in a formatted way"""
    if isinstance(metrics, tuple):
        # Handle confidence intervals
        print(f"({metrics[0]:.3f}, {metrics[1]:.3f})")
    elif isinstance(metrics, dict):
        # Handle metric dictionaries
        for metric_name, value in metrics.items():
            if isinstance(value, tuple):
                print(f"{metric_name}: ({value[0]:.3f}, {value[1]:.3f})")
            else:
                print(f"{metric_name}: {value:.3f}")
    else:
        print(f"Unknown metric type: {type(metrics)}")

def plot_metrics_comparison(city, results):
    """Plot comparison of metrics for different models"""
    models = ['persistence', 'moving_average']
    metrics = ['rmse', 'mae', 'r2']
    splits = ['validation', 'test']
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    for idx, metric in enumerate(metrics):
        # Get values and confidence intervals for each split
        values = {split: [] for split in splits}
        cis = {split: [] for split in splits}
        
        for model in models:
            for split in splits:
                metric_key = f'{split[:3]}_{metric}'
                ci_key = f'{split[:3]}_{metric}_ci'
                
                if split in results[model] and metric_key in results[model][split]:
                    values[split].append(results[model][split][metric_key])
                    cis[split].append(results[model][split][ci_key])
        
        # Only plot if we have data
        if any(values[split] for split in splits):
            # Plot bars with error bars for each split
            x = np.arange(len(models))
            width = 0.35
            
            for i, split in enumerate(splits):
                if values[split]:  # Only plot if we have values
                    axes[idx].bar(x + i*width, values[split], width,
                                yerr=[(v-ci[0], ci[1]-v) for v, ci in zip(values[split], cis[split])],
                                label=split.title(), capsize=5, alpha=0.7)
            
            axes[idx].set_xticks(x + width/2)
            axes[idx].set_xticklabels([m.replace('_', ' ').title() for m in models])
            axes[idx].set_title(f'{metric.upper()} Comparison')
            axes[idx].set_ylabel(metric.upper())
            axes[idx].legend()
        else:
            # If no data, show a message
            axes[idx].text(0.5, 0.5, 'No data available',
                         horizontalalignment='center',
                         verticalalignment='center',
                         transform=axes[idx].transAxes)
            axes[idx].set_title(f'{metric.upper()} Comparison')
    
    plt.suptitle(f'Model Performance Comparison - {city.title()}', y=1.05)
    plt.tight_layout()
    return fig

def plot_predictions(city, results, data_split='test'):
    """Plot predictions from baseline models"""
    plt.figure(figsize=(15, 8))
    
    # Get actual values
    actual = city_data[city][data_split]['AQI']
    dates = city_data[city][data_split]['Date']
    
    # Plot actual values
    plt.plot(dates, actual, label='Actual', color='black', alpha=0.6)
    
    # Plot predictions for each model
    for model_name, color in [('persistence', 'blue'), ('moving_average', 'red')]:
        predictions = results[model_name]['predictions'][data_split]
        plt.plot(dates, predictions, 
                label=model_name.replace('_', ' ').title(),
                color=color, alpha=0.6)
    
    plt.title(f'{city.title()} - {data_split.title()} Set Predictions')
    plt.xlabel('Date')
    plt.ylabel('AQI')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    return plt.gcf()

def plot_feature_importance(city, results):
    """Plot feature importance for models that support it"""
    # Get models with feature importance
    models_with_importance = {
        name: results[name]['model_config']['feature_importance']
        for name in results
        if results[name]['model_config']['feature_importance'] is not None
    }
    
    if not models_with_importance:
        print("No models with feature importance found")
        return
    
    # Create subplots
    n_models = len(models_with_importance)
    fig, axes = plt.subplots(n_models, 1, figsize=(15, 5*n_models))
    if n_models == 1:
        axes = [axes]
    
    for idx, (model_name, importance) in enumerate(models_with_importance.items()):
        # Sort features by importance
        sorted_features = sorted(importance.items(), key=lambda x: abs(x[1]), reverse=True)
        features, values = zip(*sorted_features)
        
        # Create bar plot
        bars = axes[idx].barh(features, values)
        
        # Color bars based on sign
        for bar in bars:
            if bar.get_width() < 0:
                bar.set_color('red')
            else:
                bar.set_color('blue')
        
        # Add vertical line at x=0
        axes[idx].axvline(x=0, color='black', linestyle='--', alpha=0.3)
        
        axes[idx].set_title(f'Feature Importance - {model_name.replace("_", " ").title()}')
        axes[idx].set_xlabel('Importance')
        axes[idx].set_ylabel('Features')
    
    plt.suptitle(f'Feature Importance Analysis - {city.title()}', y=1.05)
    plt.tight_layout()
    return fig

In [4]:

# Cell 4: Define baseline models
class PersistenceModel:
    def __init__(self):
        self.name = "Persistence"
    
    def fit(self, X, y):
        # No training needed for persistence model
        pass
    
    def predict(self, X):
        # For persistence model, we'll use the 1-day lag if it exists
        if 'AQI_lag_1' not in X.columns:
            raise ValueError("AQI_lag_1 feature not found in the data")
        return X['AQI_lag_1'].values

class MovingAverageModel:
    def __init__(self, window_size=7):
        self.name = "Moving Average"
        self.window_size = window_size
    
    def fit(self, X, y):
        # No training needed for moving average model
        pass
    
    def predict(self, X):
        # Create a rolling window of lag features
        lag_cols = [f'AQI_lag_{i}' for i in range(1, self.window_size + 1)]
        if not all(col in X.columns for col in lag_cols):
            raise ValueError(f"Required lag features not found in the data")
        
        # Calculate moving average
        lag_values = X[lag_cols].values
        return np.nanmean(lag_values, axis=1)  # Use nanmean to handle NaN values

In [5]:
# Cell 5: Function to evaluate baseline models
def evaluate_baseline_models(city_data, city_name):
    """Evaluate baseline models for a specific city"""
    results = {}
    
    # Initialize models
    persistence = PersistenceModel()
    moving_avg = MovingAverageModel(window_size=7)
    
    # Fit and evaluate persistence model
    persistence.fit(city_data['train'], city_data['train']['AQI'])
    val_pred_persistence = persistence.predict(city_data['val'])
    test_pred_persistence = persistence.predict(city_data['test'])

    # Calculate feature importance for persistence model
    # For persistence, the importance is 1 for lag_1 and 0 for others
    persistence_importance = {
        'AQI_lag_1': 1.0,
        'AQI_lag_2': 0.0,
        'AQI_lag_3': 0.0,
        'AQI_lag_4': 0.0,
        'AQI_lag_5': 0.0,
        'AQI_lag_6': 0.0,
        'AQI_lag_7': 0.0
    }
    
    
    # Fit and evaluate moving average model
    moving_avg.fit(city_data['train'], city_data['train']['AQI'])
    val_pred_ma = moving_avg.predict(city_data['val'])
    test_pred_ma = moving_avg.predict(city_data['test'])

     # Calculate feature importance for moving average
    # For moving average, each lag has equal importance (1/window_size)
    ma_importance = {
        f'AQI_lag_{i}': 1.0/moving_avg.window_size 
        for i in range(1, moving_avg.window_size + 1)
    }
    # Add zeros for lags beyond window size
    for i in range(moving_avg.window_size + 1, 8):
        ma_importance[f'AQI_lag_{i}'] = 0.0
    
    
    # Calculate metrics
    persistence_val_metrics = calculate_metrics(city_data['val']['AQI'].values, val_pred_persistence, 'val_')
    persistence_test_metrics = calculate_metrics(city_data['test']['AQI'].values, test_pred_persistence, 'test_')
    
    ma_val_metrics = calculate_metrics(city_data['val']['AQI'].values, val_pred_ma, 'val_')
    ma_test_metrics = calculate_metrics(city_data['test']['AQI'].values, test_pred_ma, 'test_')

    
    # Store results
    results['persistence'] = {
        'validation': persistence_val_metrics,
        'test': persistence_test_metrics,
        'predictions': {
            'val': val_pred_persistence,
            'test': test_pred_persistence
        },
        'model_config': {
            'name': 'Persistence',
            'parameters': {},
            'feature_importance': persistence_importance
        }
    }
    
    results['moving_average'] = {
        'validation': ma_val_metrics,
        'test': ma_test_metrics,
        'predictions': {
            'val': val_pred_ma,
            'test': test_pred_ma
        },
        'model_config': {
            'name': 'Moving Average',
            'parameters': {'window_size': moving_avg.window_size},
            'feature_importance': ma_importance
        }
    }
    
    return results

In [6]:
# 3. Update the ModelResultsManager class to handle ML results
class ModelResultsManager:
    def __init__(self):
        self.base_dir = "./results"
        self.ml_dir = f"{self.base_dir}/ml_models"
        self._create_directory_structure()
    
    def _create_directory_structure(self):
        """Create necessary directories"""
        os.makedirs(self.ml_dir, exist_ok=True)
        for city in ['Bengaluru', 'Chennai', 'Delhi', 'Hyderabad']:
            city_dir = f"{self.ml_dir}/{city}"
            os.makedirs(city_dir, exist_ok=True)
            os.makedirs(f"{city_dir}/visualizations", exist_ok=True)
    
    def save_ml_results(self, city, results):
        """Save ML model results"""
        city_dir = f"{self.ml_dir}/{city}"
        
        # Save results for each model
        for model_name, model_results in results.items():
            # Convert numpy arrays to lists for JSON serialization
            serializable_results = {
                'validation': model_results['validation'],
                'test': model_results['test'],
                'predictions': model_results['predictions'],
                'model_config': model_results['model_config']
            }
            
            # Save to file
            with open(f"{city_dir}/{model_name}_results.json", 'w') as f:
                json.dump(serializable_results, f, indent=4)
        
        # Update metadata
        self._update_metadata(city, results)
    
    def _update_metadata(self, city, results):
        """Update metadata file with ML model results"""
        metadata_file = f"{self.base_dir}/metadata.json"
        
        # Load existing metadata
        if os.path.exists(metadata_file):
            with open(metadata_file, 'r') as f:
                metadata = json.load(f)
        else:
            metadata = {}
        
        # Update ML results
        if 'ml_models' not in metadata:
            metadata['ml_models'] = {}
        
        metadata['ml_models'][city] = {
            'timestamp': datetime.now().isoformat(),
            'models': list(results.keys()),
            'metrics': {
                model: {
                    'validation': {
                        'rmse': results[model]['validation'][f'val_rmse'],
                        'mae': results[model]['validation'][f'val_mae'],
                        'r2': results[model]['validation'][f'val_r2']
                    },
                    'test': {
                        'rmse': results[model]['test'][f'test_rmse'],
                        'mae': results[model]['test'][f'test_mae'],
                        'r2': results[model]['test'][f'test_r2']
                    }
                }
                for model in results
            }
        }
        
        # Save updated metadata
        with open(metadata_file, 'w') as f:
            json.dump(metadata, f, indent=4)


In [7]:
# Cell 6: Define ModelResultsManager class
class ModelResultsManager:
    def __init__(self, base_dir='results'):
        self.base_dir = base_dir
        self._create_directory_structure()
    
    def _create_directory_structure(self):
        """Create necessary directories for storing results"""
        os.makedirs(f'{self.base_dir}', exist_ok=True)
        os.makedirs(f'{self.base_dir}/model_configs', exist_ok=True)
        os.makedirs(f'{self.base_dir}/predictions', exist_ok=True)
        os.makedirs(f'{self.base_dir}/performance_metrics', exist_ok=True)
        os.makedirs(f'{self.base_dir}/plots', exist_ok=True)
    
    def _convert_to_serializable(self, obj):
        """Convert numpy arrays and other non-serializable objects to serializable format"""
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, dict):
            return {k: self._convert_to_serializable(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [self._convert_to_serializable(item) for item in obj]
        elif isinstance(obj, tuple):
            return tuple(self._convert_to_serializable(item) for item in obj)
        elif isinstance(obj, (np.int64, np.int32, np.int16, np.int8,
                            np.uint64, np.uint32, np.uint16, np.uint8)):
            return int(obj)
        elif isinstance(obj, (np.float64, np.float32, np.float16)):
            return float(obj)
        elif isinstance(obj, np.bool_):
            return bool(obj)
        return obj
    
    def _update_metadata(self, city_name, model_type, results):
        """Update metadata file with latest results"""
        metadata_file = f'{self.base_dir}/metadata.json'
        
        # Load existing metadata or create new
        if os.path.exists(metadata_file):
            with open(metadata_file, 'r') as f:
                metadata = json.load(f)
        else:
            metadata = {}
        
        # Update metadata for this city and model type
        if city_name not in metadata:
            metadata[city_name] = {}
        
        metadata[city_name][model_type] = {
            'last_updated': datetime.now().isoformat(),
            'metrics_used': ['rmse', 'mae', 'r2'],
            'results': {
                'persistence': {
                    'validation': results['persistence']['validation'],
                    'test': results['persistence']['test']
                },
                'moving_average': {
                    'validation': results['moving_average']['validation'],
                    'test': results['moving_average']['test']
                }
            }
        }
        
        # Save updated metadata
        with open(metadata_file, 'w') as f:
            json.dump(metadata, f, indent=4)
    
    def save_baseline_results(self, city_name, results):
        """Save baseline model results for a city"""
        # Convert results to serializable format
        serializable_results = self._convert_to_serializable(results)
        
        # Save performance metrics
        metrics_file = f'{self.base_dir}/performance_metrics/baseline_{city_name}.json'
        with open(metrics_file, 'w') as f:
            json.dump(serializable_results, f, indent=4)
        
        # Save predictions
        pred_dir = f'{self.base_dir}/predictions/{city_name}'
        os.makedirs(pred_dir, exist_ok=True)
        
        for model_name in ['persistence', 'moving_average']:
            pred_data = {
                'validation': serializable_results[model_name]['predictions']['val'],
                'test': serializable_results[model_name]['predictions']['test']
            }
            with open(f'{pred_dir}/{model_name}_predictions.json', 'w') as f:
                json.dump(pred_data, f, indent=4)
        
        # Update metadata
        self._update_metadata(city_name, 'baseline', serializable_results)
    
    def save_plots(self, city_name, fig, plot_name):
        """Save plots for a city"""
        # Create directory for city plots if it doesn't exist
        plot_dir = f'{self.base_dir}/plots/{city_name}'
        os.makedirs(plot_dir, exist_ok=True)
        
        # Save the plot
        plot_path = f'{plot_dir}/{plot_name}.png'
        fig.savefig(plot_path, dpi=300, bbox_inches='tight')
        plt.close(fig)  # Close the figure to free memory
    
    def load_baseline_results(self, city_name):
        """Load baseline model results for a city"""
        metrics_file = f'{self.base_dir}/performance_metrics/baseline_{city_name}.json'
        if not os.path.exists(metrics_file):
            raise FileNotFoundError(f"No results found for {city_name}")
        
        with open(metrics_file, 'r') as f:
            return json.load(f)

In [8]:
# Cell 7: Run evaluation and save results
results_manager = ModelResultsManager("./results/baseline")
baseline_results = {}

for city in cities:
    print(f"\nEvaluating baseline models for {city}")
    baseline_results[city] = evaluate_baseline_models(city_data[city], city)
    
    # Print metrics
    print("\nPersistence Model:")
    print("Validation Metrics:")
    print_metrics(baseline_results[city]['persistence']['validation'])
    print("\nTest Metrics:")
    print_metrics(baseline_results[city]['persistence']['test'])
    
    print("\nMoving Average Model:")
    print("Validation Metrics:")
    print_metrics(baseline_results[city]['moving_average']['validation'])
    print("\nTest Metrics:")
    print_metrics(baseline_results[city]['moving_average']['test'])
    
    # Save results
    results_manager.save_baseline_results(city, baseline_results[city])
    
    # Generate and save plots
    fig = plot_metrics_comparison(city, baseline_results[city])
    results_manager.save_plots(city, fig, 'metrics_comparison')
    
    fig = plot_predictions(city, baseline_results[city], 'test')
    results_manager.save_plots(city, fig, 'test_predictions')

    fig = plot_feature_importance(city, baseline_results[city])
    results_manager.save_plots(city, fig, 'feature_importances')
    
    print(f"\nResults and plots saved for {city}")


Evaluating baseline models for bengaluru

Persistence Model:
Validation Metrics:
val_rmse: 17.738
val_mae: 12.563
val_r2: 0.512
val_rmse_ci: (15.407, 20.362)
val_mae_ci: (11.152, 14.210)
val_r2_ci: (0.361, 0.623)

Test Metrics:
test_rmse: 11.324
test_mae: 7.906
test_r2: 0.667
test_rmse_ci: (9.858, 12.781)
test_mae_ci: (7.142, 8.675)
test_r2_ci: (0.562, 0.746)

Moving Average Model:
Validation Metrics:
val_rmse: 21.624
val_mae: 16.289
val_r2: 0.275
val_rmse_ci: (19.245, 23.987)
val_mae_ci: (14.651, 17.989)
val_r2_ci: (0.127, 0.398)

Test Metrics:
test_rmse: 13.973
test_mae: 9.713
test_r2: 0.492
test_rmse_ci: (12.076, 16.079)
test_mae_ci: (8.797, 10.840)
test_r2_ci: (0.390, 0.577)

Results and plots saved for bengaluru

Evaluating baseline models for chennai

Persistence Model:
Validation Metrics:
val_rmse: 36.885
val_mae: 23.734
val_r2: 0.269
val_rmse_ci: (31.567, 42.050)
val_mae_ci: (20.549, 27.087)
val_r2_ci: (0.054, 0.437)

Test Metrics:
test_rmse: 27.243
test_mae: 16.415
test_r2: 0

In [9]:
class MLModelEvaluator:
    def __init__(self, city_data, city_name):
        self.city_data = city_data
        self.city_name = city_name
        self.scaler = StandardScaler()
        self.results = {}
    
    def prepare_data(self):
        """Prepare data for ML models"""
        # Get numeric columns only (excluding date, city, and AQI)
        numeric_cols = self.city_data['train'].select_dtypes(include=[np.number]).columns
        numeric_cols = [col for col in numeric_cols if col not in ['AQI']]
        
        # Separate features and target
        X_train = self.city_data['train'][numeric_cols]
        y_train = self.city_data['train']['AQI']
        
        X_val = self.city_data['val'][numeric_cols]
        y_val = self.city_data['val']['AQI']
        
        X_test = self.city_data['test'][numeric_cols]
        y_test = self.city_data['test']['AQI']
        
        # Scale features
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_val_scaled = self.scaler.transform(X_val)
        X_test_scaled = self.scaler.transform(X_test)
        
        return {
            'train': {'X': X_train_scaled, 'y': y_train},
            'val': {'X': X_val_scaled, 'y': y_val},
            'test': {'X': X_test_scaled, 'y': y_test},
            'feature_names': numeric_cols  # Removed .tolist() since it's already a list
        }
    
    def calculate_confidence_intervals(self, y_true, y_pred, metric_func, n_bootstrap=1000):
        """Calculate 95% confidence intervals using bootstrap sampling"""
        scores = []
        for _ in range(n_bootstrap):
            indices = np.random.choice(len(y_true), len(y_true), replace=True)
            score = metric_func(y_true[indices], y_pred[indices])
            scores.append(score)
        return np.percentile(scores, [2.5, 97.5])
    
    def evaluate_model(self, model, model_name, param_grid=None):
        """Evaluate a single ML model"""
        data = self.prepare_data()
        
        if param_grid:
            grid_search = GridSearchCV(
                model, param_grid, cv=5, scoring='neg_mean_squared_error',
                n_jobs=-1, verbose=1
            )
            grid_search.fit(data['train']['X'], data['train']['y'])
            best_model = grid_search.best_estimator_
            best_params = grid_search.best_params_
        else:
            best_model = model
            best_model.fit(data['train']['X'], data['train']['y'])
            best_params = {}
        
        # Make predictions
        predictions = {
            'val': best_model.predict(data['val']['X']),
            'test': best_model.predict(data['test']['X'])
        }
        
        # Calculate metrics with confidence intervals
        metrics = {}
        for split in ['val', 'test']:
            y_true = data[split]['y']
            y_pred = predictions[split]
            
            # Calculate metrics
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))
            mae = mean_absolute_error(y_true, y_pred)
            r2 = r2_score(y_true, y_pred)
            
            # Calculate confidence intervals
            rmse_ci = self.calculate_confidence_intervals(
                y_true, y_pred, lambda yt, yp: np.sqrt(mean_squared_error(yt, yp))
            )
            mae_ci = self.calculate_confidence_intervals(
                y_true, y_pred, mean_absolute_error
            )
            r2_ci = self.calculate_confidence_intervals(
                y_true, y_pred, r2_score
            )
            
            metrics[split] = {
                f'{split}_rmse': rmse,
                f'{split}_mae': mae,
                f'{split}_r2': r2,
                f'{split}_rmse_ci': rmse_ci.tolist(),
                f'{split}_mae_ci': mae_ci.tolist(),
                f'{split}_r2_ci': r2_ci.tolist()
            }
        
        # Get feature importance if available
        feature_importance = None
        if hasattr(best_model, 'feature_importances_'):
            feature_importance = dict(zip(data['feature_names'], 
                                       best_model.feature_importances_))
        elif hasattr(best_model, 'coef_'):
            feature_importance = dict(zip(data['feature_names'], 
                                       best_model.coef_))
        elif isinstance(best_model, SVR) and best_model.kernel == 'rbf':
            # For SVR with RBF kernel, use permutation importance
            perm_importance = permutation_importance(
                best_model, 
                data['test']['X'], 
                data['test']['y'],
                n_repeats=10,
                random_state=42
            )
            feature_importance = dict(zip(data['feature_names'], perm_importance.importances_mean))
        
        # Store results in baseline-compatible format
        self.results[model_name] = {
            'val': metrics['val'],
            'test': metrics['test'],
            'predictions': {
                'val': predictions['val'].tolist(),
                'test': predictions['test'].tolist()
            },
            'model_config': {
                'name': model_name,
                'parameters': best_params,
                'feature_importance': feature_importance
            }
        }
        
        return self.results[model_name]

In [10]:
def evaluate_all_ml_models(city_data, city_name):
    """Evaluate all ML models for a given city"""
    evaluator = MLModelEvaluator(city_data, city_name)
    
    # Define models and their parameter grids
    models = {
        'linear_regression': {
            'model': LinearRegression(),
            'param_grid': None
        },
        'ridge': {
            'model': Ridge(),
            'param_grid': {
                'alpha': [0.1, 1.0, 10.0, 100.0]
            }
        },
        'lasso': {
            'model': Lasso(),
            'param_grid': {
                'alpha': [0.1, 1.0, 10.0, 100.0]
            }
        },
        'random_forest': {
            'model': RandomForestRegressor(random_state=42),
            'param_grid': {
                'n_estimators': [100, 200],
                'max_depth': [None, 10, 20],
                'min_samples_split': [2, 5]
            }
        },
        'gradient_boosting': {
            'model': GradientBoostingRegressor(random_state=42),
            'param_grid': {
                'n_estimators': [100, 200],
                'learning_rate': [0.01, 0.1],
                'max_depth': [3, 5]
            }
        },
        'xgboost': {
            'model': xgb.XGBRegressor(random_state=42),
            'param_grid': {
                'n_estimators': [100, 200],
                'learning_rate': [0.01, 0.1],
                'max_depth': [3, 5]
            }
        },
        'svr': {
            'model': SVR(),
            'param_grid': {
                'kernel': ['linear', 'rbf'],
                'C': [0.1, 1.0, 10.0],
                'epsilon': [0.1, 0.2]
            }
        }
    }
    
    # Evaluate each model
    for model_name, model_config in models.items():
        print(f"\nEvaluating {model_name} for {city_name}...")
        evaluator.evaluate_model(
            model_config['model'],
            model_name,
            model_config['param_grid']
        )
        
    
    return evaluator.results


In [11]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, (np.float32, np.float64)):
            return float(obj)
        elif isinstance(obj, (np.int32, np.int64)):
            return int(obj)
        return super().default(obj)

def save_ml_results(city_name, results):
    """Save ML model results"""
    # Create results directory if it doesn't exist
    results_dir = f"./results/ml_models/{city_name}"
    os.makedirs(results_dir, exist_ok=True)
    
    # Save results for each model
    for model_name, model_results in results.items():
        # Save to file using the custom encoder
        with open(f"{results_dir}/{model_name}_results.json", 'w') as f:
            json.dump(model_results, f, indent=4, cls=NumpyEncoder)

In [18]:
def plot_metrics_comparison(city, results):
    """Plot comparison of metrics for different models"""
    # Define metrics and splits
    metrics = ['rmse', 'mae', 'r2']
    splits = ['val', 'test']
    
    # Prepare data for plotting
    models = list(results.keys())
    n_models = len(models)
    n_metrics = len(metrics)
    
    # Create figure
    fig, axes = plt.subplots(1, n_metrics, figsize=(6*n_metrics, 6))
    if n_metrics == 1:
        axes = [axes]
    
    # Plot each metric
    for i, metric in enumerate(metrics):
        ax = axes[i]
        
        # Collect values and confidence intervals
        values = {split: [] for split in splits}
        cis = {split: [] for split in splits}
        
        for model in models:
            for split in splits:
                metric_key = f'{split}_{metric}'
                ci_key = f'{split}_{metric}_ci'
                
                if metric_key in results[model][split]:
                    values[split].append(results[model][split][metric_key])
                    # Convert CI to error bar format (half-width)
                    ci = results[model][split][ci_key]
                    if isinstance(ci, (list, np.ndarray)) and len(ci) == 2:
                        ci_half_width = (ci[1] - ci[0]) / 2
                        cis[split].append(ci_half_width)
                    else:
                        cis[split].append(0)  # No error bar if CI is invalid
                else:
                    values[split].append(None)
                    cis[split].append(0)
        
        # Plot bars
        x = np.arange(n_models)
        width = 0.35
        
        for j, split in enumerate(splits):
            valid_values = [v for v in values[split] if v is not None]
            if valid_values:
                ax.bar(x + j*width, values[split], width, label=split.capitalize())
                
                # Add error bars
                for k, (val, ci) in enumerate(zip(values[split], cis[split])):
                    if val is not None:
                        ax.errorbar(x[k] + j*width, val, yerr=ci, 
                                  fmt='none', color='black', capsize=5)
        
        # Customize plot
        ax.set_xlabel('Models')
        ax.set_ylabel(metric.upper())
        ax.set_title(f'{metric.upper()} Comparison')
        ax.set_xticks(x + width/2)
        ax.set_xticklabels(models, rotation=45, ha='right')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig


def plot_predictions(city, results, data_split, actual_values):
    """Plot actual vs predicted values for all models"""
    # Create figure
    fig, ax = plt.subplots(figsize=(15, 8))
    
    # Plot actual values
    ax.plot(actual_values, label='Actual', color='black', alpha=0.7)
    
    # Plot predictions for each model
    for model_name, model_results in results.items():
        predictions = model_results['predictions'][data_split]
        ax.plot(predictions, label=model_name, alpha=0.7)
    
    # Customize plot
    ax.set_xlabel('Time')
    ax.set_ylabel('AQI')
    ax.set_title(f'Actual vs Predicted AQI - {data_split.capitalize()}')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

def plot_feature_importance(city, results):
    """Plot feature importance for models that support it"""
    # Get models with feature importance
    models_with_importance = {
        name: results[name]['model_config']['feature_importance']
        for name in results
        if results[name]['model_config']['feature_importance'] is not None
    }
    
    if not models_with_importance:
        print("No models with feature importance found")
        return
    
    # Create subplots
    n_models = len(models_with_importance)
    fig, axes = plt.subplots(n_models, 1, figsize=(15, 5*n_models))
    if n_models == 1:
        axes = [axes]
    
    for idx, (model_name, importance) in enumerate(models_with_importance.items()):
        print(f"\nDebugging {model_name}:")
        print("Feature importance structure:", importance)
        
        # Handle different formats of feature importance
        if isinstance(importance, dict):
            # For models with dictionary format (most models)
            sorted_features = sorted(importance.items(), key=lambda x: abs(x[1]), reverse=True)
            features, values = zip(*sorted_features)
            
            # Convert values to float if they're arrays
            try:
                values = [float(v) if hasattr(v, '__len__') else v for v in values]
            except Exception as e:
                print(f"Error converting values: {e}")
                print("Original values:", values)
                # Try alternative conversion
                values = [float(v[0]) if hasattr(v, '__len__') else float(v) for v in values]
        else:
            # For models with array format (SVR with linear kernel)
            features = ['PM2.5']  # Assuming PM2.5 is the target
            values = [float(importance[0])]  # Take the first value from the array
        
        # Create bar plot
        bars = axes[idx].barh(features, values)
        
        # Color bars based on sign
        for bar in bars:
            if bar.get_width() < 0:
                bar.set_color('red')
            else:
                bar.set_color('blue')
        
        # Add vertical line at x=0
        axes[idx].axvline(x=0, color='black', linestyle='--', alpha=0.3)
        
        axes[idx].set_title(f'Feature Importance - {model_name.replace("_", " ").title()}')
        axes[idx].set_xlabel('Importance')
        axes[idx].set_ylabel('Features')
    
    plt.suptitle(f'Feature Importance Analysis - {city.title()}', y=1.05)
    plt.tight_layout()
    return fig


def save_ml_visualizations(city, results, city_data):
    """Save all visualizations for ML models"""
    # Create visualization directory
    viz_dir = f"./notebooks/results/ml_models/{city}/visualizations"
    os.makedirs(viz_dir, exist_ok=True)
    
    # Generate and save plots
    fig = plot_metrics_comparison(city, results)
    fig.savefig(f"{viz_dir}/metrics_comparison.png", bbox_inches='tight', dpi=300)
    plt.close(fig)
    
    fig = plot_feature_importance(city, results)
    fig.savefig(f"{viz_dir}/feature_importance.png", bbox_inches='tight', dpi=300)
    plt.close(fig)

    
    # Plot predictions for each split
    for split in ['val', 'test']:
        # Get actual values from the data
        actual_values = city_data[split]['AQI'].values
        fig = plot_predictions(city, results, split, actual_values)
        fig.savefig(f"{viz_dir}/predictions_{split}.png", bbox_inches='tight', dpi=300)
        plt.close(fig)

In [13]:
# 1. First, let's load the preprocessed data for each city
def load_city_data(city_name):
    """Load preprocessed data for a specific city"""
    data_dir = "../data/processed"
    city_data = {}
    
    for split in ['train', 'val', 'test']:
        file_path = f"{data_dir}/{city_name}/{split}.csv"
        # print(file_path)
        if os.path.exists(file_path):
            city_data[split] = pd.read_csv(file_path)
            city_data[split]['date'] = pd.to_datetime(city_data[split]['Date'])
    
    return city_data

def run_ml_analysis():
    """Run ML analysis for all cities"""
    cities = ['bengaluru', 'chennai', 'delhi', 'hyderabad']
    
    for city in cities:
        print(f"\nProcessing {city}...")
        
        # Load city data
        city_data = load_city_data(city)
        if not city_data:
            print(f"No data found for {city}")
            continue
        
        # Evaluate all ML models
        print("Evaluating ML models...")
        ml_results = evaluate_all_ml_models(city_data, city)
        
        # Save results
        print("Saving results...")
        save_ml_results(city, ml_results)
        
        # Generate and save visualizations
        print("Generating visualizations...")
        save_ml_visualizations(city, ml_results, city_data)
        
        # Print summary metrics
        print("\nModel Performance Summary:")
        for model_name, results in ml_results.items():
            print(f"\n{model_name.replace('_', ' ').title()}:")
            for split in ['val', 'test']:
                print(f"{split.title()} Set:")
                print(f"  RMSE: {results[split][f'{split}_rmse']:.2f}")
                print(f"  MAE: {results[split][f'{split}_mae']:.2f}")
                print(f"  R²: {results[split][f'{split}_r2']:.2f}")


In [14]:
# 3. Update the ModelResultsManager class to handle ML results
class ModelResultsManager:
    def __init__(self):
        self.base_dir = "./results"
        self.ml_dir = f"{self.base_dir}/ml_models"
        self._create_directory_structure()
    
    def _create_directory_structure(self):
        """Create necessary directories"""
        os.makedirs(self.ml_dir, exist_ok=True)
        for city in ['Bengaluru', 'Chennai', 'Delhi', 'Hyderabad']:
            city_dir = f"{self.ml_dir}/{city}"
            os.makedirs(city_dir, exist_ok=True)
            os.makedirs(f"{city_dir}/visualizations", exist_ok=True)
    
    def save_ml_results(self, city, results):
        """Save ML model results"""
        city_dir = f"{self.ml_dir}/{city}"
        
        # Save results for each model
        for model_name, model_results in results.items():
            # Convert numpy arrays to lists for JSON serialization
            serializable_results = {
                'validation': model_results['validation'],
                'test': model_results['test'],
                'predictions': model_results['predictions'],
                'model_config': model_results['model_config']
            }
            
            # Save to file
            with open(f"{city_dir}/{model_name}_results.json", 'w') as f:
                json.dump(serializable_results, f, indent=4)
        
        # Update metadata
        self._update_metadata(city, results)
    
    def _update_metadata(self, city, results):
        """Update metadata file with ML model results"""
        metadata_file = f"{self.base_dir}/metadata.json"
        
        # Load existing metadata
        if os.path.exists(metadata_file):
            with open(metadata_file, 'r') as f:
                metadata = json.load(f)
        else:
            metadata = {}
        
        # Update ML results
        if 'ml_models' not in metadata:
            metadata['ml_models'] = {}
        
        metadata['ml_models'][city] = {
            'timestamp': datetime.now().isoformat(),
            'models': list(results.keys()),
            'metrics': {
                model: {
                    'validation': {
                        'rmse': results[model]['validation'][f'val_rmse'],
                        'mae': results[model]['validation'][f'val_mae'],
                        'r2': results[model]['validation'][f'val_r2']
                    },
                    'test': {
                        'rmse': results[model]['test'][f'test_rmse'],
                        'mae': results[model]['test'][f'test_mae'],
                        'r2': results[model]['test'][f'test_r2']
                    }
                }
                for model in results
            }
        }
        
        # Save updated metadata
        with open(metadata_file, 'w') as f:
            json.dump(metadata, f, indent=4)


In [19]:
# Initialize results manager
results_manager = ModelResultsManager()

# Run ML analysis
run_ml_analysis()


Processing bengaluru...
Evaluating ML models...

Evaluating linear_regression for bengaluru...

Evaluating ridge for bengaluru...
Fitting 5 folds for each of 4 candidates, totalling 20 fits

Evaluating lasso for bengaluru...
Fitting 5 folds for each of 4 candidates, totalling 20 fits

Evaluating random_forest for bengaluru...
Fitting 5 folds for each of 12 candidates, totalling 60 fits

Evaluating gradient_boosting for bengaluru...
Fitting 5 folds for each of 8 candidates, totalling 40 fits

Evaluating xgboost for bengaluru...
Fitting 5 folds for each of 8 candidates, totalling 40 fits

Evaluating svr for bengaluru...
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Saving results...
Generating visualizations...

Debugging linear_regression:
Feature importance structure: {'PM2.5': np.float64(11.245920919896736), 'PM10': np.float64(9.566417657086019), 'NO': np.float64(4.182867582259376), 'NO2': np.float64(-0.1882131403400237), 'NOx': np.float64(0.8313367302709116), 'NH3': n